# Configuring Jupyter On Amazon Deep Learning AMI

Using this tutorial you will be able to install and run Jupyter using https on a Amazon Deep Learning AMI. AWS now also offers a Ubuntu version of the Deep Learning AMI, most of the following steps are compatible with both versions of the Deep Learning AMI.

### Launch an EC2 Instance With The Amazon Deep Learning AMI

1. Launch an instance using the Amazon Deep Learning AMI using the AWS console:
https://aws.amazon.com/marketplace/pp/B01M0AXXQB
2. Navigate to the AWS Management Console:

2. Or Launch the Amazon Deep Learning AMI from a CloudFormation Script:
https://github.com/awslabs/deeplearning-cfn/blob/master/cfn-template/deeplearning.template

3. Choose an instance type sutiable for deep learning. Common choices are GPU backed instances such as g2 and p2 instance types, both of which have dedicated GPU's. 

In [6]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://s3-us-west-2.amazonaws.com/mxnetjupytersetup/Untitled.jpg")

Next, we are going to create a security group. This security group contains some specific rules to allow you to access the Jupyter server from your browser with HTTP/HTTPS. A sample ruleset used for this tutorial is below. However, for an enterprise grade enviroment, you should probably secure the source ip's to only known and trusted ips.

In [7]:
 from IPython.display import HTML, display

 data = [["<b>Type</b>","<b>Protocol</b>","<b>Port Range</b>", "<b>Source</b>"],
         ["HTTPS","TCP",443,"0.0.0.0/0"],
         ["HTTP","TCP",80,"0.0.0.0/0"],
         ["SSH","TCP",22,"0.0.0.0/0"],
         ["Custom TCP Rule", "TCP", 8888, "0.0.0.0/0"]
         ]

 display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
 ))

Type,Protocol,Port Range,Source
HTTPS,TCP,443,0.0.0.0/0
HTTP,TCP,80,0.0.0.0/0
SSH,TCP,22,0.0.0.0/0
Custom TCP Rule,TCP,8888,0.0.0.0/0


You can add these rules through the AWS console while setting up the instance as follows:

In [8]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://s3-us-west-2.amazonaws.com/mxnetjupytersetup/SecurityGroup.jpg")

Optionally, you can create the security group using the AWS CLI beforehand and attach it to the instance. Keep in mind you need to update these commands to suit your own needs. In this tutorial we are using the latest Deep Learning AMI 2.0 (ami-dfb13ebf) in the us-west-2 region. The CLI commands for this are as follows:

In [ ]:

#Setup the AWS CLI on your instance
aws configure
#Enter your access key and secret key 

# Create Security Group
aws ec2 create-security-group --group-name JupyterSG --description "JupyterSG"
#Add SG ruleset
aws ec2 authorize-security-group-ingress --group-name JupyterSG --protocol tcp --port 8888 --cidr 0.0.0.0/0
aws ec2 authorize-security-group-ingress --group-name JupyterSG --protocol tcp --port 22 --cidr 0.0.0.0/0
aws ec2 authorize-security-group-ingress --group-name JupyterSG --protocol tcp --port 443 --cidr 0.0.0.0/0
aws ec2 authorize-security-group-ingress --group-name JupyterSG --protocol tcp --port 80 --cidr 0.0.0.0/0

#Launch AWS instance
aws ec2 run-instances --image-id ami-dfb13ebf --count 1 --instance-type p2.xlarge --key-name <YOUR_KEY_NAME> --security-groups JupyterSG

### Setup The Environment On Your New Instance

SSH into the instance using the instances DNS name

In [ ]:
ssh -i <PATH_TO_PEM> ec2-user@ec2-xx-xxx-xx-xxx.eu-west-1.compute.amazonaws.com

Use the AMI's pre installed Anaconda3 environment managment tool to activate a default environment named "root" with the following command:

In [ ]:
source src/anaconda3/bin/activate root

Verify that your environment is using the correct binaries with the following commands:

In [ ]:
which python
~/src/anaconda3/bin/python
which  jupyter
~/src/anaconda3/bin/jupyter

If they are not pointing to the correct binaries change the PATH definition to the following:

In [ ]:
PATH=$HOME/src/anaconda3/bin:$PATH:$HOME/.local/bin:$HOME/bin
export PATH

### Configure The Jupyter Server To Use HTTPS

Now that you have your AMI launched and your environment setup correctly we can begin to generate the configuration for Jupyter. In this tutorial, we will setup the configuration using HTTPS. If you want to use regular HTTP you can skip the following steps.

In [ ]:
jupyter notebook --generate-config
key=$(python -c "from notebook.auth import passwd; print(passwd())")

Create a directory for your server certificates then use OpenSSL to self-sign a certificate

In [ ]:
cd ~
mkdir certs
cd certs
certdir=$(pwd)
openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout JupyterKey.key -out JupyterCert.pem

In [ ]:
cd ~
sed -i "1 a\
c = get_config()\\
c.NotebookApp.certfile = u'$certdir/JupyterCert.pem'\\
c.NotebookApp.keyfile = u'$certdir/JupyterKey.key'\\
c.NotebookApp.ip = '*'\\
c.NotebookApp.open_browser = False\\
c.NotebookApp.password = u'$key'\\
c.NotebookApp.port = 8888" .jupyter/jupyter_notebook_config.py

### Running The Jupyter Server

Now that Jupyter is configured we can start the server. You can either start the server directly, or use session management to make sure that the server keeps running even after you log out of the instance. The steps for configuring the server to run with the session management tool screen are below:

In [ ]:
screen -S jupyter
mkdir notebook
cd notebook
jupyter notebook

The above steps create a notebook directory and starts the Jupyter server with a default notebook.

You can now access your Jupyter server in your browser with the following URL: https://your-instances-public-ip:8888


In [9]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="	https://s3-us-west-2.amazonaws.com/mxnetjupytersetup/JupyterNotebook.jpg")

Now that you have launched the AWS Deep Learning AMI, and configured Jupyter you can run the next tutorials for learning how to use Jupyter, and many other MXNet tutorials. Check out the MXNet GitHub Repo ( https://github.com/dmlc/mxnet-notebooks) for tutorials that make it easy to start using Jupyter for computer vision, natural language processing, and recommendation systems. 

### Importing MXNet Notebooks
The following steps teach you how to pull notebooks from GitHub and how to sync your newly create Jupyter notebooks to S3 for backup. You can run these AWS CLI commands from your instance command prompt in order to store and sync your notebook files to S3

To get notebooks from GitHub on your instance us the command line run the following command from the Jupyter notebooks directory on your instance filesystem. 

In [ ]:
cd /home/ec2-user/notebook
git clone https://github.com/dmlc/mxnet-notebooks/

The command above pulls the CNN MXNet tutorial from the AWS MXNet github repo and places it into Jupyter as a notebook you can use to train a CNN. You can now load this notebook from the Jupyter console.

In [10]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://s3-us-west-2.amazonaws.com/mxnetjupytersetup/MxNetNotebook.jpg")

In order to use the sample notebooks you need to make sure all the notebook dependencies are installed in your enviroment, the steps for doing so are below:

In [ ]:
sudo pip install -U scikit-learn
cd /home/ec2-user/src/anaconda3/bin
sudo ./conda install graphviz

MXNet comes pre-installed on the AWS Deep Learning AMI, now by going into the mxnet-notebooks folder you can run all sort of deep learning tutorials, from handwritting reccognition to a LSTM neural network. Check them out and have fun!

### Backing Up Notebooks to S3
Backing your notebooks up to S3 is a good way to ensure that you do not lose all the hard work you have put into creating them. The steps for doing so are below:

Create a S3 bucket in the AWS console, then use the following commands to backup your notebooks to S3:

In [ ]:
aws s3 cp /home/ec2-user/notebook/ s3://BucketName --recursive

You now have a backup of all of your notebooks in S3 that you can pull or push to from the AWS CLI on your AWS Deep Learning AMI instance.

### Embedding Files in a Notebook
Throughout the course of this tutorial we have added images into the notebook through python code. There are many ways to add files into a notebook. Check out the python code above the images in this post and try and post your own image inside of a notebook using the format. Below we will discuss serveral other file types you can add to your notebooks.

Below we import two diffrent auto files that have been converted from text to speech by AWS Polly into our Jupyter notebook and allow the user of the notebook to play them with an Ipython.display.Audio object. This is a simple use case but it can be very handy later on for deep learning tasks that require audio, for example, natural language processing engines. 

In [21]:
import IPython
IPython.display.display(IPython.display.Audio(url="https://s3.amazonaws.com/bardiapollylambda/35591d25a681531497481c12aed2a7c283c42773_0.mp3"))
IPython.display.display(IPython.display.Audio(url="https://s3.amazonaws.com/bardiapollylambda/5feea3d3b888aff62974d5f30f1d7142b6c96e49_0.mp3"))

After going through this tutorial you should be able to stand up a fully functional Jupyter notebook on the AWS Deep Learning AMI and use MXNet to run and create your own deep learning architectures. A few sample excercises to test your knowlage are as follows:
* Deploy an AWS Deep Learning instance
* Setup Jupyter on your own AWS Deep Learning instance 
* Create your first Jupyter notebook
* Import an image from S3 into your notebook
* Import an audio file from S3 into your notebook

Work Hard, Have Fun, and, Make History!